In [ ]:
import sys
sys.path.append('../')

from model import *
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# MIMIC - Triage

This notebook runs the analysis on the MIMIC emergency data by leveraging experts' agreement
1. Explore a model build on data ignoring experts 
2. Compute agreement between experts using influence function
3. Retrain the model on the set of labels for which experts strongly agree

The current analysis uses multi layer perceptrons in a single train / test split.

In [ ]:
data_set = "../data/triage_semi_synthetic.csv" # Data file
selective_labels = False # Is it a case of selective labels (only observe the outcome for patients filtered by nurse: D == 1)

### Data

Reopen the data created with the notebook in `data/`

To use with your data, you will need to change the following line to open a file with:
- `X` covariates
- `H` associated (human) experts 
- `D` their decision for each case
- `Y` observed outcome
- `Yc` a concept 

In [ ]:
triage = pd.read_csv(data_set, index_col = [0, 1])
covariates, target, nurses = triage.drop(columns = ['D', 'Y1', 'Y2', 'YC', 'acuity', 'nurse']), triage[['D', 'Y1', 'Y2', 'YC']], triage['nurse']

Split data in a 80% train, 20% test

In [ ]:
cov_train, cov_test, tar_train, tar_test, nur_train, nur_test = train_test_split(covariates, target, nurses, test_size = 0.2, random_state = 42)

### Modelling

In [ ]:
# Model's characteristics
params = {'layers': [100]} # If = [] equivalent to a simple logistic regression

##### 1. Train on decision

This model models the nurse decision based on covariates

In [ ]:
model = BinaryMLP(**params)
model = model.fit(cov_train, tar_train['D'], nur_train, platt_calibration=True)
predictions = model.predict(cov_test)

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], predictions)

In [ ]:
# Yc performance
roc_auc_score(tar_test['YC'], predictions)

In [ ]:
# D performance
roc_auc_score(tar_test['D'], predictions)

##### 2. Agreement computation 

Measure of agreeability are estimated in a cross validation fashion on the train set.

In [ ]:
# Fold evaluation of influences
folds, predictions, influence = influence_cv(BinaryMLP, cov_train, tar_train['D'], nur_train, params = params, l1_penalties = [0.001, 0.01, 0.1, 1])

In [ ]:
# Compute metrics agreeability
center_metric, opposing_metric = compute_agreeability(influence)

In [ ]:
# Analyze confident points
rho = 0.05 # Control which point to consider from a confience point of view

In [ ]:
high_conf = (predictions > (1 - rho)) | (predictions < rho)

In [ ]:
pd.Series(center_metric[high_conf]).plot.hist(bins = 100)

In [ ]:
pd.Series(opposing_metric[high_conf]).plot.hist(bins = 100)

In [ ]:
# Amalgation parameters
pi_1 = 2.5 # Control criterion on centre mass metric
pi_2 = 0.95 # Control criterion on opposing metric
tau = 1.0  # Balance between observed and expert labels

In [ ]:
# Apply criteria on amalgamation
high_agr = (center_metric > pi_1) & (opposing_metric > pi_2) & high_conf
high_agr_correct = ((predictions - tar_train['D']).abs() < rho) & high_agr

In [ ]:
# Create amalgamated labels
tar_train['Ya'] = tar_train['Y1'].astype(int)
tar_train['Ya'][high_agr_correct] = (1 - tau) * tar_train['Y1'][high_agr_correct] \
                                    + tau * tar_train['D'][high_agr_correct]

In [ ]:
index_amalg = (tar_train['D'] | high_agr_correct) if selective_labels else tar_train['D'].isin([0, 1])
print("Use: {:.2f} % of data".format(100 * index_amalg.mean()))

##### 3. Updated model

In [ ]:
model = BinaryMLP(**params)
model = model.fit(cov_train[index_amalg], tar_train[index_amalg]['Ya'], nur_train[index_amalg])
predictions = model.predict(cov_test)

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], predictions)

In [ ]:
# Yc performance
roc_auc_score(tar_test['YC'], predictions)

In [ ]:
# D performance
roc_auc_score(tar_test['D'], predictions)

##### 4. Train on observed data

In [ ]:
index_observed = tar_train['D'] if selective_labels else tar_train['D'].isin([0, 1])

In [ ]:
model = BinaryMLP(**params)
model = model.fit(cov_train[index_observed], tar_train['Y1'][index_observed], nur_train[index_observed])
predictions = model.predict(cov_test)

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], predictions)

In [ ]:
# Yc performance
roc_auc_score(tar_test['YC'], predictions)

In [ ]:
# D performance
roc_auc_score(tar_test['D'], predictions)

##### 5. Hybrid alternative

- Leverage human model in the amalgamation set
- Leverage outcome model on non amalgamation set

Models need to be retrain on their respective subsets and calibrated to ensure to mix

In [ ]:
predictions = pd.Series(0, index = cov_test.index)

In [ ]:
# Compute which test points are part of A for test set
predictions_test, influence_test = influence_estimate(BinaryMLP, cov_train, tar_train['D'], nur_train, cov_test, params = params, l1_penalties = [0.001, 0.01, 0.1, 1])
center_metric, opposing_metric = compute_agreeability(influence_test)
high_conf_test = (predictions_test > (1 - rho)) | (predictions_test < rho)
high_agr_test = (center_metric > pi_1) & (opposing_metric > pi_2) & high_conf_test
high_agr_correct_test = ((predictions_test - tar_test['D']).abs() < rho) & high_agr_test

In [ ]:
# Retrain a model on amalgamation points only and calibrate: Rely on humans
model = BinaryMLP(**params)
model = model.fit(cov_train[high_agr_correct], tar_train['D'][high_agr_correct], nur_train[high_agr_correct], platt_calibration = True)
predictions.loc[high_agr_correct_test] = model.predict(cov_test.loc[high_agr_correct_test])

In [ ]:
# Retrain a model on non almagamation only and calibrate: Rely on observed
model = BinaryMLP(**params)
model = model.fit(cov_train[index_observed], tar_train['Y1'][index_observed], nur_train[index_observed], platt_calibration = True)
predictions.loc[~high_agr_correct_test] = model.predict(cov_test.loc[~high_agr_correct_test])

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], predictions)

In [ ]:
# Yc performance
roc_auc_score(tar_test['YC'], predictions)

In [ ]:
# D performance
roc_auc_score(tar_test['D'], predictions)